In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figsizeO = [8,4.7]
figsizeS = [8,4]
plt.rcParams['figure.figsize'] = figsizeO
import os
resultDir = '.' + os.path.sep + 'results' + os.path.sep

In [ ]:
plotOutputPath = '.' + os.path.sep + 'plots' + os.path.sep
show = False
dpi=300
plt.rcParams.update({'font.size': 11})

In [ ]:
comparableNodes = 56 / 4
linethicknes = 1.5

In [ ]:
def extractName(name):
    name = name.split(' ')[0].split(':')
    return name[1] if len(name) > 1 else name[0]

In [ ]:
def extractDuration(time):
    time = time.split(' ')
    seconds = 0.0
    for t in time:
        unit = ''.join([s for s in t if s.isalpha()])
        amount = float(t[:-len(unit)])
        if unit == 'ms':
            seconds += amount / 1000
        elif unit == 's':
            seconds += amount
        elif unit == 'm':
            seconds += amount * 60
        elif unit == 'h':
            seconds += amount * 3600
        elif unit == 'd':
            seconds += amount * 86400
        else:
            raise Exception('Unknown unit', unit, time)
    return seconds

In [ ]:
def extractCPU(cpu):
    return float(cpu[:-1])

In [ ]:
def extractStorageInMB(storage):
    storage = storage.split(' ')
    if(len(storage) != 2):
        raise Exception('Unknown unit', storage)
    amount = float(storage[0]) 
    unit = storage[1]
    if unit == 'MB':
        return amount
    elif unit == 'GB':
        return amount * 1024
        

### Nextflow report shows:
- realtime
- peak_rss
- %CPU
- rchar
- wchar

In [ ]:
aggregations = ['mean','min','max','median','std','sum','count']
byteToMb = 1024 * 1024
msPerMinute = 60 * 1000
msPerHour = msPerMinute * 60

valuesToExtract=[
    ('duration',msPerMinute,'min'),('realtime', msPerMinute,'min'),
    ('%cpu',1,'%'),('%mem', 1,'%'),
    ('rss',byteToMb,'mb'),('vmem', byteToMb,'mb'),
    ('peak_rss',byteToMb,'mb'),('peak_vmem', byteToMb,'mb'),
    ('read_bytes',byteToMb,'mb'),('write_bytes', byteToMb,'mb'),
    ('rchar',byteToMb,'mb'),('wchar', byteToMb,'mb')
                ]

In [ ]:
def loadTrace(path):
    print(path)
    return pd.read_csv(path,sep=';')  

In [ ]:
def reduceToMetric(trace, metric, divisor = 1):
    shortedTrace = trace[['process',metric]]
    
    l = len(shortedTrace)
    shortedTrace = shortedTrace[shortedTrace[metric] != '-']
    if l != len(shortedTrace):
        print(metric,"shortened!!!")
        shortedTrace[metric] = shortedTrace[metric].astype('float')
    
    global processes
    processes = shortedTrace['process'].unique()
    shortedTrace[metric] = shortedTrace[metric] / divisor
    result = shortedTrace.groupby('process').agg( aggregations )
    result.columns = result.columns.get_level_values(1)
    return result

def getColumn(trace, column):
    t = trace[column]
    t = t[t != '-']
    return t.astype('float')

def extractMetrics(trace):
    results = {}
    
    #stages
    preprocesses = trace[trace['process'].str.contains(':preprocess')]
    merges = trace[trace['process'].str.contains(':merge')]
    higherLevel = trace[trace['process'].str.contains('higherLevel:')]
    checkResults = trace[trace['process'].str.contains('checkResults')]
    
    for v in valuesToExtract:
        results[v[0]] = reduceToMetric(trace, v[0], v[1])
        
    results['totalRuntime'] = (getColumn(checkResults,'submit').min() - getColumn(trace,'submit').min()) / msPerMinute
    results['sumRuntime'] = (getColumn(trace,'realtime').sum() - getColumn(checkResults,'realtime').sum()) / msPerHour        
    results['cpuRuntime'] = ((getColumn(trace,'realtime').multiply(getColumn(trace,'%cpu') / 100)).sum() - (getColumn(checkResults,'realtime').multiply(getColumn(checkResults,'%cpu') / 100)).sum()) / msPerHour   
    results['requestedCPURuntime'] = (getColumn(trace,'realtime').multiply(getColumn(trace,'cpus')).sum() - (getColumn(checkResults,'realtime').multiply(getColumn(checkResults,'cpus'))).sum()) / msPerHour 
    

    
    
    #preprocess
    
    results['stagePreprocess-Duration'] = (getColumn(merges,'submit').min() - getColumn(preprocesses,'submit').min()) / msPerMinute
    results['stagePreprocess-RealtimeSum'] = getColumn(preprocesses,'realtime').sum() / msPerHour    
    results['stagePreprocess-CpuRuntime'] = (getColumn(preprocesses,'realtime').multiply(getColumn(preprocesses,'%cpu') / 100)).sum() / msPerHour 
    
    #merge
    
    results['stageMerge-Duration'] = (getColumn(higherLevel,'submit').min() - getColumn(merges,'submit').min()) / msPerMinute
    results['stageMerge-RealtimeSum'] = getColumn(merges,'realtime').sum() / msPerHour
    results['stageMerge-CpuRuntime'] = (getColumn(merges,'realtime').multiply(getColumn(merges,'%cpu') / 100)).sum() / msPerHour 
    
    results['stagePreprocessMerge-Duration'] = results['stagePreprocess-Duration'] + results['stageMerge-Duration']
    
    #higher level
    
    results['stageHigherLevel-Duration'] = (getColumn(checkResults,'submit').min() - getColumn(higherLevel,'submit').min()) / msPerMinute
    results['stageHigherLevel-RealtimeSum'] = getColumn(higherLevel,'realtime').sum() / msPerHour
    results['stageHigherLevel-CpuRuntime'] = (getColumn(higherLevel,'realtime').multiply(getColumn(higherLevel,'%cpu') / 100)).sum() / msPerHour
    
    results['stageHigherLevel-rchar'] = getColumn(higherLevel,'rchar').sum() / byteToMb
    results['stageHigherLevel-wchar'] = getColumn(higherLevel,'wchar').sum() / byteToMb
        
    return results
    

In [ ]:
def loadData():
    results = {}
    nodes = [x for x in os.listdir(resultDir)]
    for node in nodes:
        if node == 'original':
            continue
        n = 'nf' if node == 'nf' else int(node)
        results[n] = {}
        runs = [x for x in os.listdir(resultDir + node)]
        for run in runs:
            trace = loadTrace(resultDir + node + os.path.sep + run + os.path.sep + 'trace.txt' )
            #rename stage, if name is wrong in logs
            trace['process'] = trace['process'].str.replace('level2processing:','higherLevel:')
            results[n][int(run)] = extractMetrics(trace)
    return results

data = loadData()
data['original'] = {
    1 : {
    'stagePreprocessMerge-Duration' : (326 * 60000 + 47 * 1000 + 913) / msPerMinute,
    'stageHigherLevel-Duration' : (29 * 60000 + 40 * 1000 + 160
    #mosaicking
    + 10 * 1000 + 903
    #pyramids
    + 46 * 1000 + 532) / msPerMinute,
    'totalRuntime' : (357 * 60000 + 33 * 1000 + 941) / msPerMinute
    },
    2 : {
    'stagePreprocessMerge-Duration' : (332 * 60000 + 27 * 1000 + 727) / msPerMinute,
    'stageHigherLevel-Duration' : (25 * 60000 + 17 * 1000 + 349
    #mosaicking
    + 10 * 1000 + 883
    #pyramids
    + 49 * 1000 + 208) / msPerMinute,
    'totalRuntime' : (358 * 60000 + 53 * 1000 + 651) / msPerMinute
    },
    3 : {
    'stagePreprocessMerge-Duration' : (325 * 60000 + 0 * 1000 + 367) / msPerMinute,
    'stageHigherLevel-Duration' : (29 * 60000 + 8 * 1000 + 621
    #mosaicking
    + 10 * 1000 + 863
    #pyramids
    + 46 * 1000 + 970) / msPerMinute,
    'totalRuntime' : (355 * 60000 + 15 * 1000 + 808) / msPerMinute
    }
}

In [ ]:
def changeName(s):
    stage = None
    if 'preprocessmerge' in s.lower():
        stage = 'Preprocessing + Merging'
    elif 'preprocess' in s.lower():
        stage = 'Preprocessing'
    elif 'merge' in s.lower():
        stage = 'Merging'
    elif 'higherlevel' in s.lower():
        stage = 'Higher Level'
    elif 'totalruntime' in s.lower():
        stage = 'Total Runtime'
    elif 'sumruntime' in s.lower():
        stage = 'Cumulated Runtime'
    elif 'requestedcpuruntime' in s.lower():
        stage = 'Requested CPU Runtime'
    elif 'cpuruntime' in s.lower():
        stage = 'CPU Runtime'
    elif 'min' in s.lower():
        stage = 'Minimum'
    elif 'mean' in s.lower():
        stage = 'Average'
    elif 'median' in s.lower():
        stage = 'Median'
    elif 'max' in s.lower():
        stage = 'Maximum'
        
    if stage is None:
        return s
        
    workflow = None
    if s.endswith('NF'):
        workflow = 'HPS'
    elif s.endswith('Original'):
        workflow = 'HPS Original'
    
    if workflow is not None:
        return stage + ' ' + workflow
    return stage

def sortHelper(s):
    if 'preprocessmerge' in s.lower():
        return '3-' + s
    elif 'preprocess' in s.lower():
        return '1-' + s
    elif 'merge' in s.lower():
        return '2-' + s
    elif 'higherlevel' in s.lower():
        return '4-' + s
    elif 'min' in s.lower():
        return '1-' + s
    elif 'median' in s.lower():
        return '2-' + s
    elif 'mean' in s.lower():
        return '3-' + s
    elif 'max' in s.lower():
        return '4-' + s
    return '5-' + s
    
def labelLegend(labels,handles):
    labels,handles = zip(*sorted(zip(labels, handles), key=lambda t: sortHelper(t[0])))
    labels = list(labels)
    handles = list(handles)
    labels = list(map(changeName,labels))
    
    bringToEnd = ['Optimal Scaling']
    for e in bringToEnd:
        if e in labels:
            index = labels.index(e)
            l = labels.pop(index)
            labels.append(l)
            h = handles.pop(index)
            handles.append(h)
    
    return labels,handles

In [ ]:
marker = {
    'mean' : ('black','--','1'),'min' : ('b','-.','2'),'max' : ('g', ':','3'),'median': ('r','solid','4'), 'sum' : ('black','solid','x')
}

def rearangeDict(dictionary, fields):
    results = {}
    for run in dictionary:
        for agg in dictionary[run]:
            if agg in fields:
                for pro in dictionary[run][agg]:
                    aggC = agg
                    d = dictionary[run][aggC][pro]
                    if int(dictionary[run]['count'][pro]) == 1:
                        aggC = 'mean'
                        if agg == 'sum':
                            d = None
                    a = results.get(pro,{})
                    results[pro] = a
                    b = a.get(aggC,{})
                    a[aggC] = b 
                    b[run] = d
    return results

def plotOneProcess(dictionary, metric, process, unit, outPath = plotOutputPath, figsize = figsizeO, nCol = 1):
    plt.rcParams["figure.figsize"] = figsize
    for agg in dictionary:
        
        nfValue = None
        if 'nf' in dictionary[agg]:
            nfValue = dictionary[agg]['nf']
            del dictionary[agg]['nf']
        
        lists = sorted(dictionary[agg].items()) # sorted by key, return a list of tuples
        x, y = zip(*lists)
        
        if all(v is None for v in y):
            return
        
        plt.xticks(np.unique(x),np.array(x))
        
        plt.plot(list(map(int,x)), list(map(float,y)),label=agg, linestyle = marker[agg][1], marker = marker[agg][2], linewidth=linethicknes, c = marker[agg][0])
        
        xLabels = np.array(x)
        
        if nfValue is not None:
            plt.scatter(comparableNodes, nfValue, label = agg + 'NF', marker=marker[agg][2], linewidth=linethicknes, c = marker[agg][0])
            x = list(x)
            x.append(14)
            x.sort()
            xLabels = x.copy()
            xLabels[x.index(14)] = 'HPS'
        
    
    name = "avg. cpu usage" if metric == "%cpu" else metric
    plt.title(name + ' of ' + process)
    plt.xlabel('# of nodes')
    plt.ylabel(name + ' in ' + unit)
    
    plt.xticks(np.array(x),xLabels)
    
    if len(dictionary) > 1:
        plt.legend()
        handles, labels = plt.gca().get_legend_handles_labels()
        # sort both labels and handles by labels
        labels, handles = labelLegend(labels,handles)
        plt.gca().legend(handles, labels, ncol = nCol)
        
    plt.grid(color='grey', linestyle='-', linewidth=.1)
    
    plt.savefig(outPath + process.replace(':','-') + '-' + metric.replace('%','percent_') + '.png', bbox_inches='tight', dpi = dpi)
    plt.title(None)
    plt.savefig(outPath + process.replace(':','-') + '-' + metric.replace('%','percent_') + '.pdf', bbox_inches='tight', dpi = dpi)
    plt.rcParams["figure.figsize"] = figsizeO
    if show:
        plt.show()
    else:
        plt.close()

def extractData(data, metric):
    allData = {}
    for d in data:
        results = []
        for run in data[d]:
            if metric not in data[d][run]:
                break
            results.append( data[d][run][metric] )
        if len(results) > 0:
            results = pd.concat(results).groupby('process').agg(['median'])   
            results.columns = results.columns.get_level_values(0)
            allData[d] = results .to_dict()
    return allData

def plot(data,metric,unit):
    allData = extractData(data,metric)
    oneExecution = rearangeDict(allData,['mean','min','max','median']) 
    sumExecution = rearangeDict(allData,['sum'])
    for proc in oneExecution:
        plotOneProcess(oneExecution[proc],metric,proc,unit)   
        if unit != '%':
            plotOneProcess(sumExecution[proc],'sum_' + metric,proc,unit)
    

In [ ]:
os.makedirs( plotOutputPath + 'overleaf', exist_ok=True)

allData = extractData(data,'realtime')
oneExecution = rearangeDict(allData,['mean','min','max','median']) 
plotOneProcess(oneExecution['preprocessing:preprocess'],'realtime','preprocessing:preprocess','min', plotOutputPath + 'overleaf' + os.path.sep, figsize = figsizeS, nCol = 2)  

In [ ]:


for v in valuesToExtract:
    plot(data,v[0],v[2])

In [ ]:
os.makedirs(plotOutputPath + 'stages', exist_ok=True)

def plotSingleMetrics(data,metric,unit,plot=True,optimumLine = False, efficiency = False, hps = False, ylim = None, outPath = plotOutputPath + 'stages'  + os.path.sep, figsize = figsizeO, title = "duration"):
    plt.rcParams["figure.figsize"] = figsize
    allData = {}
    for nodes in data:
        results = []
        for run in data[nodes]:
            if metric in data[nodes][run]:
                results.append(data[nodes][run][metric])
        if len(results) > 0:
            allData[nodes] = np.median(results)
    print(metric,allData)
    
    originalValue = None
    if 'original' in allData:
        originalValue = allData['original']
        del allData['original']
        
    nfValue = None
    if 'nf' in allData:
        nfValue = allData['nf']
        del allData['nf']
    
    lists = sorted(allData.items()) # sorted by key, return a list of tuples
    x, y = zip(*lists)
    y0 = y[0] 
    if efficiency :
        y = (y0 / np.array(x)) / np.array(y)
    else:
        y = y / y0
    
    if '-' in metric:
        markerTuple = markerMultiple[metric[:metric.index('-')]]
    else:
        markerTuple = ('black','solid','x','v')
    
    #plt.plot(list(map(int,x)), list(map(float,y)), label = metric,linewidth=linethicknes, marker=markerTuple[2],linestyle=markerTuple[1],color=markerTuple[0])
    plt.plot(list(map(int,x)), list(map(float,y)), label = metric,linewidth=linethicknes,linestyle=markerTuple[1],color=markerTuple[0])
    plt.title(metric)
    plt.xlabel('# of nodes')
    
    plt.ylabel( title )
    
    xLabels = np.array(x)
    
    if originalValue is not None:
        print('original')
        xVal = (y0 / comparableNodes) / originalValue if efficiency else originalValue / y0
        plt.scatter(comparableNodes, xVal, label = metric + 'Original',linewidth=linethicknes, marker=markerTuple[3],color=markerTuple[0])
        
    if nfValue is not None:
        xVal = (y0 / comparableNodes) / nfValue if efficiency else nfValue / y0
        plt.scatter(comparableNodes, xVal, label = metric + 'NF',linewidth=linethicknes, marker=markerTuple[2],color=markerTuple[0])
        
    if hps or nfValue is not None:
        x = list(x)
        x.append(14)
        x.sort()
        xLabels = x.copy()
        xLabels[x.index(14)] = 'HPS'
        
    
    if optimumLine:
        plt.plot(np.array(list(map(int,x))), 1. / np.array(list(map(int,x))), label = 'Optimal Scaling', color='red',linewidth=0.5, zorder=-100) 
        
    plt.grid(color='grey', linestyle='-', linewidth=.1)
    
    plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()]) 
    
    
    plt.xticks(x,xLabels)
    if ylim is not None:
        plt.ylim(ylim)
    
    if plot:
        plt.legend()
        handles, labels = plt.gca().get_legend_handles_labels()
        # sort both labels and handles by labels
        labels, handles = labelLegend(labels,handles)
        plt.gca().legend(handles, labels)
        plt.savefig(outPath + metric + '.png', bbox_inches='tight', dpi = dpi)
        plt.title(None)
        plt.savefig(outPath + metric + '.pdf', bbox_inches='tight', dpi = dpi)
        if show:
            plt.show()
        else:
            plt.close()
            
    plt.rcParams["figure.figsize"] = figsizeO
            
def plotMultipleMetrics(metrics,endsWith,data,imageName,optimumLine = True, efficiency = False, hps = False, ylim = None, outPath = plotOutputPath + 'stages'  + os.path.sep, figsize = figsizeO, nCol = 1, title = "duration"):
    first = optimumLine
    for sm in metrics:
        if(sm[0].endswith(endsWith)):
            plotSingleMetrics(data,sm[0],sm[1],False,first, efficiency, hps, ylim,figsize=figsize,title=title)
            first = False
            
    plt.legend(loc = 'lower left' if efficiency else 'upper right')
    
    handles, labels = plt.gca().get_legend_handles_labels()
    # sort both labels and handles by labels
    labels, handles = labelLegend(labels,handles)
    plt.gca().legend(handles, labels, ncol = nCol)
    
    plt.title(imageName)
    plt.savefig(outPath + imageName + '.png', bbox_inches='tight', dpi = dpi)
    plt.title(None)
    plt.savefig(outPath + imageName + '.pdf', bbox_inches='tight', dpi = dpi)
    if show:
        plt.show()
    else:
        plt.close()
    

singleMetrics = [
    ('totalRuntime','min',True, 'total runtime in comparison to one node'),
    ('sumRuntime','h',False, 'cum. runtime in comparison to one node'),
    ('cpuRuntime','h',False, 'cum. CPU hours in comparison to one node'),
    ('requestedCPURuntime','h',False, 'requested CPU hours in comparison to one node'),
    ('stagePreprocessMerge-Duration','min'),
    ('stageHigherLevel-Duration','min'),
    ('stageHigherLevel-RealtimeSum','h'),
    ('stageHigherLevel-CpuRuntime','h'),
    ('stagePreprocess-Duration','min'),
    ('stagePreprocess-RealtimeSum','h'),
    ('stagePreprocess-CpuRuntime','h'),
    ('stageMerge-Duration','min'),
    ('stageMerge-RealtimeSum','h'),
    ('stageMerge-CpuRuntime','h')
]  

markerMultiple = {
    'stageMerge' : ('r',':','1'),'stageHigherLevel' : ('g','solid','2','v'),'stagePreprocess' : ('black','--','3'),'stagePreprocessMerge': ('b','-.','4','^')
}

for sm in singleMetrics[:4]:
    plotSingleMetrics(data,sm[0],sm[1], optimumLine=sm[2], title = sm[3] )

plotMultipleMetrics(singleMetrics[4:],'Sum',data,'stagesSumRuntime', False, title = "cum. runtime in comparison to one node")
plotMultipleMetrics(singleMetrics[4:],'Duration',data,'stagesDurationEfficiency', False, True, True,(0,1.01), nCol = 2, title = "efficiency in comparison to one node")
plotMultipleMetrics(singleMetrics[4:],'Duration',data,'stagesDuration', True, False, True, title = "duration in comparison to one node")
plotMultipleMetrics(singleMetrics[4:],'CpuRuntime',data,'stagesCpuRuntime', False, title = "cum. CPU hours in comparison to one node")


plotSingleMetrics(data,singleMetrics[0][0],singleMetrics[0][1], optimumLine=singleMetrics[0][2], outPath=plotOutputPath + 'overleaf'  + os.path.sep, figsize=figsizeS, title = singleMetrics[0][3])

plotMultipleMetrics(singleMetrics[4:],'Sum',data,'stagesSumRuntime', False, outPath=plotOutputPath + 'overleaf'  + os.path.sep, figsize=figsizeS, title = "cum. runtime in comparison to one node")

In [ ]:
import shutil
os.makedirs(plotOutputPath + 'overleaf', exist_ok=True)
files = ['higherLevel-processHigherLevel-percent_cpu']
filesStages = ['stagesDuration','stagesDurationEfficiency']
for f in files:
    shutil.copy(plotOutputPath + f + '.pdf', plotOutputPath + 'overleaf')
for f in filesStages:
    shutil.copy(plotOutputPath + 'stages' + os.path.sep + f + '.pdf', plotOutputPath + 'overleaf')

In [ ]:
#Important metrics:
def getRuntime(node,metric):
    #if str(node) == '14':
    #    a = getRuntime(10,metric)
    #    b = getRuntime(15,metric)
    #    return np.interp(14, [10,15], [a,b])
        
    results = []
    for run in data[node]:
        results.append(data[node][run][metric])
    return np.median(results)

def isNumber(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def findNodesForY(y, metric):
    nodes = []
    for n in data:
        if isNumber(n):
            nodes.append(int(n))
    nodes.sort()
    values = []
    for n in nodes:
        values.append(getRuntime(n,metric))
    index = 0
    for k in range(1, len(values)):
        if( values[ k - 1 ] >= y and values[ k ] <= y ):
            index = k
            break
    if(index == 0):
        raise Exception('No value found')
    m = ((values[k]-values[k-1])/(nodes[k]-nodes[k-1]))
    r = nodes[k-1] + (y - values[k-1]) / m
    return r

print()
print('Total Runtime')

wfDuration1Node = getRuntime(1,'totalRuntime')
wfDuration21Node = getRuntime(21,'totalRuntime')
wfDuration14Node = getRuntime(14,'totalRuntime')
wfDurationNFNode = getRuntime('nf','totalRuntime')
wfDurationOriginalNode = getRuntime('original','totalRuntime')

print('Runtime in min 1:',wfDuration1Node,'21:',wfDuration21Node,'14:',wfDuration14Node,'NF',wfDurationNFNode, 'Orig', wfDurationOriginalNode)
print('Runtime 1 -> 21',wfDuration1Node / wfDuration21Node,'x')
print('Runtime NF vs 14',1 - wfDurationNFNode / wfDuration14Node,'%')
print('Runtime Original higher 21',wfDurationOriginalNode / wfDuration21Node - 1,'%')
print('Runtime NF vs Original',wfDurationNFNode / wfDurationOriginalNode - 1,'%')

print()
print('Preprocessing')
print()

preprocessDuration1Node = getRuntime(1,'stagePreprocess-Duration')
print('stagePreprocess-Duration 1 node', preprocessDuration1Node, 'm')
preprocessDuration21Node = getRuntime(21,'stagePreprocess-Duration')
print('stagePreprocess-Duration 21 node', preprocessDuration21Node, 'm')
preprocessDuration14Node = getRuntime(14,'stagePreprocess-Duration')
print('stagePreprocess-Duration 14 node', preprocessDuration14Node, 'm')
preprocessDurationNFNode = getRuntime('nf','stagePreprocess-Duration')
print('stagePreprocess-Duration NF node', preprocessDurationNFNode, 'm')
print('stagePreprocess-Duration 1 -> 21 nodes improvement', preprocessDuration1Node / preprocessDuration21Node, 'x')
print('stagePreprocess-Duration 21 efficiency', (preprocessDuration1Node / 21) / preprocessDuration21Node, '%')

stagePreprocessRealtimeSum14Node = getRuntime(14,'stagePreprocess-RealtimeSum')
stagePreprocessRealtimeSumNFNode = getRuntime('nf','stagePreprocess-RealtimeSum')

print('stagePreprocess-RealtimeSum NF overhead', (stagePreprocessRealtimeSumNFNode / stagePreprocessRealtimeSum14Node) - 1, '%')

print()
print('Merge')
print()

mergeDuration1Node = getRuntime(1,'stageMerge-Duration')
mergeDuration21Node = getRuntime(21,'stageMerge-Duration')
mergeDuration21NodeEfficiency = ((mergeDuration1Node / 21) / mergeDuration21Node)
print('stageMerge-Duration 21 efficiency', mergeDuration21NodeEfficiency / 21 * 100, '%')

print()
print('Preproces - Merge')
print()

preprocessmergeDuration1Node = getRuntime(1,'stagePreprocessMerge-Duration')
preprocessmergeDuration14Node = getRuntime(14,'stagePreprocessMerge-Duration')
preprocessmergeDuration21Node = getRuntime(21,'stagePreprocessMerge-Duration')
preprocessmergeDurationOriginalNode = getRuntime('original','stagePreprocessMerge-Duration')
preprocessMergeDuration14NodeEfficiency = ((preprocessmergeDuration1Node / 14) / preprocessmergeDuration14Node)
preprocessMergeDurationOriginalNodeEfficiency = ((preprocessmergeDuration1Node / 14) / preprocessmergeDurationOriginalNode)
print('stagePreprocessMerge-Duration 14 efficiency', preprocessMergeDuration14NodeEfficiency * 100, '%')
print('stagePreprocessMerge-Duration Original efficiency', preprocessMergeDurationOriginalNodeEfficiency * 100, '%')
print('stagePreprocessMerge-Duration outperform original', findNodesForY(preprocessmergeDurationOriginalNode,'stagePreprocessMerge-Duration'), 'nodes')
print('stagePreprocessMerge-Duration 21 earlier than Original', (1 - preprocessmergeDuration21Node / preprocessmergeDurationOriginalNode) * 100, '%')

print()
print('Higher Level')
print()

higherLevelDuration1Node = getRuntime(1,'stageHigherLevel-Duration')
print('higherLevelDuration1Node-Duration 1 node', higherLevelDuration1Node, 'min')

higherLevelDuration14Node = getRuntime(14,'stageHigherLevel-Duration')
higherLevelDuration21Node = getRuntime(21,'stageHigherLevel-Duration')
higherLevelDurationNFNode = getRuntime('nf','stageHigherLevel-Duration')
higherLevelDurationOriginalNode = getRuntime('original','stageHigherLevel-Duration')

print('stageHigherLevel runtime with 1 nodes higher 21 node', higherLevelDuration1Node / higherLevelDuration21Node, 'x')
higherLevelDuration21NodeEfficiency = ((higherLevelDuration1Node / 21) / higherLevelDuration21Node)
print('stagePreprocessMerge-Duration 21 efficiency', higherLevelDuration21NodeEfficiency * 100, '%')



higherLevelRChar1Node = getRuntime(1,'stageHigherLevel-rchar') / 1024
higherLevelWChar1Node = getRuntime(1,'stageHigherLevel-wchar') / 1024
print('Higher Level I/O: r:',higherLevelRChar1Node,'gb','w:', higherLevelWChar1Node,'gb')


higherLevelRealtimeSum1Node = getRuntime(1,'stageHigherLevel-RealtimeSum')
higherLevelRealtimeSum20Node = getRuntime(20,'stageHigherLevel-RealtimeSum')

print('stagePreprocessMerge-RuntimeSum Increase 1->20', higherLevelRealtimeSum20Node / higherLevelRealtimeSum1Node * 100, '%')

higherLevelDuration14NodeEfficiency = ((higherLevelDuration1Node / 14) / higherLevelDuration14Node)
higherLevelDurationOriginalNodeEfficiency = ((higherLevelDuration1Node / 14) / higherLevelDurationOriginalNode)
higherLevelDurationNFNodeEfficiency = ((higherLevelDuration1Node / 14) / higherLevelDurationNFNode)
print('higherLevelMerge-Duration 14 efficiency', higherLevelDuration14NodeEfficiency * 100, '%')
print('higherLevelMerge-Duration Original efficiency', higherLevelDurationOriginalNodeEfficiency * 100, '%')
print('higherLevelMerge-Duration NF efficiency', higherLevelDurationNFNodeEfficiency * 100, '%')
print('stageHigherLevel runtime with 21 nodes vs Original node', higherLevelDuration21Node / higherLevelDurationOriginalNode, '%')

print()
print('Discussion')
print()


totalDuration21NodeEfficiency = ((wfDuration1Node / 21) / wfDuration21Node)
print('21 Node total efficiency', totalDuration21NodeEfficiency * 100, '%')
print('21 Node preprocessing share', preprocessDuration21Node / wfDuration21Node * 100, '%')
higherLevelDuration21NodeEfficiency = ((higherLevelDuration1Node / 21) / higherLevelDuration21Node)
print('higherLevelMerge-Duration 21 efficiency', higherLevelDuration21NodeEfficiency * 100, '%')


print()
print('Conclusion')
print()

print('total runtime outperform original', findNodesForY(preprocessmergeDurationOriginalNode,'totalRuntime'), 'nodes')